In [ ]:
!python -m spacy download en_core_web_trf
!pip install spacy
!pip install spacy-transformers

In [ ]:
import spacy
import spacy_transformers

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import pandas as pd
import requests
from tqdm import tqdm
from collections import Counter
import json

In [ ]:
lit_df = pd.read_csv('literature.csv')
lit_df = lit_df[lit_df['author']!='William Shakespeare']
lit_df

In [ ]:
nlp = spacy.load("en_core_web_trf")

In [ ]:
lit_df[lit_df['author']=='Jane Austen'].iloc[0]['url']
lit_df[lit_df['author']=='Jane Austen'].iloc[0]['title']

'Sense and Sensibility'

In [ ]:
def extract_adjective_noun_pairs(url):
  page = requests.get(url)
  clean_text = ('\r\n\r\n').join([x for x in page.text.split('\r\n\r\n') if 'gutenberg' not in x.lower()]).split('START: FULL LICENSE')[0]
  split_text = clean_text.split('\n')
  
  results = []
  for entry in tqdm(split_text):
    doc = nlp(entry)
    for token in doc:
        if (token.dep_ == 'amod') & (token.head.pos_ == 'NOUN'):
          results.append(('amod',token.text,token.head.text))
  
  counter = Counter(results)

  return counter.most_common()

test = extract_adjective_noun_pairs('https://www.gutenberg.org/files/161/161-0.txt')

100%|██████████| 12693/12693 [24:40<00:00,  8.57it/s]


In [ ]:
def create_network_data(data, title, source='adjective',target='noun', threshold=3):
  nodes = [{"id":x[0][1],"type":source} for x in data if x[1] > threshold] + [{"id":x[0][2],"type":target} for x in data if x[1] > threshold]
  nodes = list({v['id']:v for v in nodes}.values())
  links = [{"source":x[0][1],"target":x[0][2],"count":x[1]} for x in data if x[1] > threshold]
  
  dumps = json.dumps({"nodes":nodes,"links":links}, indent=4)  

  with open(f"/content/drive/MyDrive/NLP_2023/pos_network_data/{title}-{source}-{target}.json", "w") as outfile:
      outfile.write(dumps)
 
  return nodes


In [ ]:
def extract_all(df):
  for idx in range(len(df)):
    title = df.iloc[idx]['title'].lower().replace(' ','-')
    data = extract_adjective_noun_pairs(df.iloc[idx]['url'])
    create_network_data(data, title, source='adjective',target='noun', threshold=3)
    print(title)
  
extract_all(lit_df)

100%|██████████| 4455/4455 [09:32<00:00,  7.79it/s]


a-tale-of-the-tub


100%|██████████| 22306/22306 [49:57<00:00,  7.44it/s]


the-journal-to-stella


100%|██████████| 9199/9199 [18:39<00:00,  8.22it/s]


...

100%|██████████| 1/1 [00:00<00:00, 1295.74it/s]


heart-of-darkness


100%|██████████| 11300/11300 [25:01<00:00,  7.53it/s]


lord-jim


 18%|█▊        | 3148/17185 [06:34<27:49,  8.41it/s]

In [ ]:

doc = nlp("He ran quickly up the stairs")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])


He nsubj ran VERB []
ran ROOT ran VERB [He, quickly, up]
quickly advmod ran VERB []
up prep ran VERB [stairs]
the det stairs NOUN []
stairs pobj up ADP [the]


In [ ]:
def extract_root_pairs(url):
  
  page = requests.get(url)
  clean_text = ('\r\n\r\n').join([x for x in page.text.split('\r\n\r\n') if 'gutenberg' not in x.lower()]).split('START: FULL LICENSE')[0]
  split_text = clean_text.split('\n')
  
  results = []
  for entry in tqdm(split_text):
    doc = nlp(entry)
    for token in doc:
      if (token.dep_ == 'nsubj') & (token.pos_ == 'PROPN') & (token.head.pos_ == 'VERB'):
          result = ('nsubj',token.text,token.head.text,[x.text for x in token.rights])
          print(result)
          results.append(result)
  
  counter = Counter(results)

  return counter.most_common()

test = extract_root_pairs('https://www.gutenberg.org/files/161/161-0.txt')

In [ ]:
test

[(('amod', 'old', 'gentleman'), 108),
 (('amod', 'old', 'lady'), 65),
 (('amod', 'young', 'lady'), 56),
 (('amod', 'young', 'man'), 33),
 (('amod', 'old', 'man'), 33),
 (('amod', 'young', 'gentleman'), 28),
 (('amod', 'old', 'woman'), 20),
 (('amod', 'great', 'deal'), 20),
 (('amod', 'white', 'waistcoat'), 19),
 (('amod', 'public', 'house'), 19),
 (('amod', 'long', 'time'), 18),
 (('amod', 'cocked', 'hat'), 18),
 (('amod', 'young', 'woman'), 17),
 (('amod', 'low', 'voice'), 16),
 (('amod', 'few', 'words'), 14),
 (('amod', 'right', 'hand'), 13),
 (('amod', 'poor', 'boy'), 12),
 (('amod', 'next', 'morning'), 11),
 (('amod', 'little', 'room'), 11),
 (('amod', 'next', 'day'), 11),
 (('amod', 'dear', 'lady'), 11),
 (('amod', 'first', 'time'), 10),
 (('amod', 'good', 'deal'), 10),
 (('amod', 'great', 'number'), 10),
 (('amod', 'same', 'time'), 10),
 (('amod', 'young', 'friend'), 10),
 (('amod', 'last', 'night'), 9),
 (('amod', 'old', 'women'), 9),
 (('amod', 'great', 'coat'), 9),
 (('amod', 

In [ ]:
def create_df(pos,ents,top_k=30):
  ents = [[x[0][0],x[0][1],x[1],'ENT'] for x in ents][:top_k]
  pos = [[x[0][0],x[0][1],x[1], 'POS'] for x in pos][:top_k]
  df = pd.concat([pd.DataFrame(pos,columns=['text','type','count','category']),pd.DataFrame(ents,columns=['text','type','count','category'])])
  df = df[(df['type']!='PUNCT')&(df['type']!='PART')&(df['type']!='SPACE')]

  return df

In [ ]:
# df = create_df(pos,ents)
# df

In [ ]:
def nest_df(df, threshold=2):
  pos_list = []
  ent_list = []
  df = df[(df['count']>threshold)]

  pos_cats = list(set(df[df['category']=='POS']['type']))
  ent_cats = list(set(df[df['category']=='ENT']['type']))

  for pos in pos_cats:
    pos_list.append({'name':pos,'children':[]})
  for ent in ent_cats:
    ent_list.append({'name':ent,'children':[]})

  for pos in pos_list:
    df_reduce = df[df['type']==pos['name']]
    word_collection = []
    for idx in range(len(df_reduce)):
      word_collection.append({'name':df_reduce['text'].iloc[idx], 'value':int(df_reduce['count'].iloc[idx])})
    pos['children'] = word_collection

  for ent in ent_list:
    df_reduce = df[df['type']==ent['name']]
    word_collection = []
    for idx in range(len(df_reduce)):
      word_collection.append({'name':df_reduce['text'].iloc[idx], 'value':int(df_reduce['count'].iloc[idx])})
    ent['children'] = word_collection

  return pos_list, ent_list

In [ ]:
#nest_df(df, threshold=2)

In [ ]:
def create_hierarchical_data(lit_df, threshold=0):
  ## PERIOD
  ## AUTHOR
  ## BOOK
  ## ENTS / POS
  ## POS Types / Ent Types
  ## Alphabetical ?
  data = {}

  periods = list(set(lit_df['period']))

  data['name'] = 'literature'
  data['children'] = []

  for period in periods:
    data['children'].append({'name':period, 'children':[]})
    
  for entry in data['children']:
    period = entry['name']
    authors = list(set(lit_df[lit_df['period']==period]['author']))
    for auth in authors:
      entry['children'].append({'name': auth, 'children':[]})

  for entry in data['children']:
    for author in entry['children']:
      books = list(set(lit_df[lit_df['author']==author['name']]['title']))
      for book in books:
        author['children'].append({'name': book, 'children':[]})

  for entry in tqdm(data['children']):
    for author in entry['children']:
      for book in author['children']:
        url = lit_df[lit_df['title']==book['name']]['url'].iloc[0]
        pos, ents = break_down_text(url)
        df = create_df(pos,ents)
        pos_types, ent_types = nest_df(df, threshold)

        book['children'].append({'name':'parts_of_speech', 'children':pos_types})
        book['children'].append({'name':'named_entities', 'children':ent_types})
        
          
  dumps = json.dumps(data, indent=4)  

  with open("/content/drive/MyDrive/NLP_2023/literature_hierarchy.json", "w") as outfile:
      outfile.write(dumps)

  return data

create_hierarchical_data(lit_df, threshold=5)
  

Streaming output truncated to the last 5000 lines.
 52%|█████▏    | 8544/16557 [25:52<26:19,  5.07it/s]